In [1]:
import warnings
warnings.filterwarnings('ignore')

# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [2]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [3]:
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42：退会前月の退会顧客データを作成しよう

In [4]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


### ノック43：継続顧客のデータを作成しよう

In [6]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [7]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201809,IK234815,7,7.0,XXX,C02,F,2018-05-12,NaN,CA2,...,デイタイム,7500.0,入会費半額,6.727273,7.0,9.0,5.0,1.0,2019-04-30,11.0
1,201810,OA207651,7,10.0,XXXX,C01,M,2018-09-13,NaN,CA1,...,オールタイム,10500.0,通常,8.571429,8.0,10.0,7.0,1.0,2019-04-30,7.0
2,201811,HD484088,4,6.0,XXXXX,C01,M,2015-10-01,NaN,CA1,...,オールタイム,10500.0,通常,4.333333,4.0,6.0,3.0,1.0,2019-04-30,42.0
3,201807,HD513936,4,4.0,XXXXXX,C01,M,2015-12-01,NaN,CA1,...,オールタイム,10500.0,通常,4.916667,5.0,7.0,2.0,1.0,2019-04-30,40.0
4,201808,GD876521,9,8.0,XXX,C01,F,2017-09-01,NaN,CA1,...,オールタイム,10500.0,通常,5.916667,5.5,9.0,2.0,1.0,2019-04-30,19.0


In [8]:
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,max,mean,median,membership_period,min,name,price,routine_flg,start_date,年月
0,2019-04-30,CA2,入会費半額,C02,デイタイム,7,7.0,IK234815,NaN,NaN,...,9.0,6.727273,7.0,11.0,5.0,XXX,7500.0,1.0,2018-05-12,201809
1,2019-04-30,CA1,通常,C01,オールタイム,7,10.0,OA207651,NaN,NaN,...,10.0,8.571429,8.0,7.0,7.0,XXXX,10500.0,1.0,2018-09-13,201810
2,2019-04-30,CA1,通常,C01,オールタイム,4,6.0,HD484088,NaN,NaN,...,6.0,4.333333,4.0,42.0,3.0,XXXXX,10500.0,1.0,2015-10-01,201811
3,2019-04-30,CA1,通常,C01,オールタイム,4,4.0,HD513936,NaN,NaN,...,7.0,4.916667,5.0,40.0,2.0,XXXXXX,10500.0,1.0,2015-12-01,201807
4,2019-04-30,CA1,通常,C01,オールタイム,9,8.0,GD876521,NaN,NaN,...,9.0,5.916667,5.5,19.0,2.0,XXX,10500.0,1.0,2017-09-01,201808


### ノック44：予測する月の在籍期間を作成しよう

In [9]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,median,membership_period,min,name,price,routine_flg,start_date,年月,period,now_date
0,2019-04-30,CA2,入会費半額,C02,デイタイム,7,7.0,IK234815,NaN,NaN,...,7.0,11.0,5.0,XXX,7500.0,1.0,2018-05-12,201809,3,2018-09-01
1,2019-04-30,CA1,通常,C01,オールタイム,7,10.0,OA207651,NaN,NaN,...,8.0,7.0,7.0,XXXX,10500.0,1.0,2018-09-13,201810,0,2018-10-01
2,2019-04-30,CA1,通常,C01,オールタイム,4,6.0,HD484088,NaN,NaN,...,4.0,42.0,3.0,XXXXX,10500.0,1.0,2015-10-01,201811,37,2018-11-01
3,2019-04-30,CA1,通常,C01,オールタイム,4,4.0,HD513936,NaN,NaN,...,5.0,40.0,2.0,XXXXXX,10500.0,1.0,2015-12-01,201807,31,2018-07-01
4,2019-04-30,CA1,通常,C01,オールタイム,9,8.0,GD876521,NaN,NaN,...,5.5,19.0,2.0,XXX,10500.0,1.0,2017-09-01,201808,11,2018-08-01


### ノック45：欠損値を除去しよう

In [10]:
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1               246
customer_id             0
end_date             2842
exit_date            2842
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
年月                      0
period                  0
now_date                0
dtype: int64

In [11]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1                 0
customer_id             0
end_date             2648
exit_date            2648
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
年月                      0
period                  0
now_date                0
dtype: int64

### ノック46：文字列型の変数を処理できるように整形しよう

In [12]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,入会費半額,デイタイム,F,7.0,1.0,3,0.0
1,通常,オールタイム,M,10.0,1.0,0,0.0
2,通常,オールタイム,M,6.0,1.0,37,0.0
3,通常,オールタイム,M,4.0,1.0,31,0.0
4,通常,オールタイム,F,8.0,1.0,11,0.0


In [13]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,7.0,1.0,3,0.0,1,0,0,0,1,0,1,0
1,10.0,1.0,0,0.0,0,0,1,1,0,0,0,1
2,6.0,1.0,37,0.0,0,0,1,1,0,0,0,1
3,4.0,1.0,31,0.0,0,0,1,1,0,0,0,1
4,8.0,1.0,11,0.0,0,0,1,1,0,0,1,0


In [14]:
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,7.0,1.0,3,0.0,1,0,0,1,1
1,10.0,1.0,0,0.0,0,0,1,0,0
2,6.0,1.0,37,0.0,0,0,1,0,0
3,4.0,1.0,31,0.0,0,0,1,0,0
4,8.0,1.0,11,0.0,0,0,1,0,1


### ノック47：決定木を用いて退会予測モデルを作成してみよう

In [15]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0.
 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1.
 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0.
 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1.
 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0.
 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0.
 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0.

In [16]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
1994,0.0,0.0
466,1.0,1.0
1086,0.0,0.0
769,1.0,1.0
1267,0.0,0.0


### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

In [17]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.908745247148289


In [18]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.908745247148289
0.9803548795944234


In [19]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9239543726235742
0.9252217997465145


### ノック49：モデルに寄与している変数を確認しよう

In [20]:
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.328187
1,routine_flg,0.116204
2,period,0.552029
3,campaign_name_入会費半額,0.002277
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.000186
7,gender_F,0.001116


### ノック50：顧客の退会を予測しよう

In [21]:
count_1 = 3
routing_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

In [22]:
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [23]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0.01587302 0.98412698]]
